In [ ]:
#!pip3 uninstall pymysql



In [ ]:
!pip3 install pymysql==0.9.3

In [ ]:
import pymysql

# Connect to AWS RDS 

In [ ]:
connection = pymysql.connect( 'database-1.cbrnhoat32p7.us-east-1.rds.amazonaws.com', 'myuser', 'myPass05')
cursor = connection.cursor()
print(cursor)

In [ ]:
cursor.execute("select version()")


In [ ]:
data = cursor.fetchone()

In [ ]:
data

In [ ]:
sql = '''select database collections'''

In [ ]:
cursor.connection.commit()

In [ ]:
sql = '''use collections'''  # use the db
cursor.execute(sql)

In [ ]:
#show the tables in RDs
sql = '''
show tables
'''
cursor.execute(sql)


In [ ]:
cursor.fetchall()

In [ ]:
sql = '''select * from items'''
cursor.execute(sql)
cursor.connection.commit()